In [ ]:
import sys
from pathlib import Path
import json

import pandas as pd
import numpy as np
import geopandas as gp
import matplotlib

DIR = Path('..')
sys.path.append(str(DIR))

import gtfs_kit as gk

DATA_DIR = DIR/'data/'

%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# List feed

path = DATA_DIR/'cairns_gtfs.zip'
gk.list_feed(path)

In [ ]:
# Read feed and describe

path = DATA_DIR/'cairns_gtfs.zip'
feed = gk.read_feed(path, dist_units='km')
feed.describe()

In [ ]:
# Validate

feed.validate()

In [ ]:
# Append shape_dist_traveled column to stop times

display(feed.stop_times.head().T)

feed = feed.append_dist_to_stop_times()
feed.stop_times.head().T

In [ ]:
# Choose study dates

week = feed.get_first_week()
dates = [week[4], week[6]]  # First Friday and Sunday
dates

In [ ]:
# Compute feed time series

trip_stats = feed.compute_trip_stats()
trip_stats.head().T
fts = feed.compute_feed_time_series(trip_stats, dates, freq='6H')
fts

In [ ]:
gk.downsample(fts, freq='12H')


In [ ]:
# Compute feed stats for first week

feed_stats = feed.compute_feed_stats(trip_stats, week)
feed_stats

In [ ]:
# Compute route time series

rts = feed.compute_route_time_series(trip_stats, dates, freq='12H')
rts

In [ ]:
# Slice time series

inds = ['service_distance', 'service_duration', 'service_speed']
rids = ['110-423', '110N-423']

rts.loc[:, (inds, rids)]

In [ ]:
# Slice again by cross-section

rts.xs(rids[0], axis="columns", level=1)

In [ ]:
# Compute trip locations for every hour

rng = pd.date_range('1/1/2000', periods=24, freq='H')
times = [t.strftime('%H:%M:%S') for t in rng]
loc = feed.locate_trips(dates[0], times)
loc.head()

In [ ]:
# Build a route timetable

route_id = feed.routes['route_id'].iat[0]
feed.build_route_timetable(route_id, dates).T

In [ ]:
# Compute screen line counts

path = DATA_DIR/'cairns_screen_lines.geojson'
lines = gp.read_file(path)

display(lines)

feed.compute_screen_line_counts(lines, dates)

In [ ]:
# Map routes

rids = feed.routes.route_id.loc[2:4]
feed.map_routes(rids, include_stops=True)
